In [43]:
import requests
import pandas as pd 
import numpy as np
from os import listdir
from os.path import isfile, join
import pathlib
import urllib
import urllib.request
import zipfile
import io

def buildUrl(year, month):
    return "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "{0:0=2d}".format(month) + "-capitalbikeshare-tripdata.zip"

def downloadFile(url):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    ourPath =  str(pathlib.Path().absolute()) + "/data/bikeShare/monthlyData/"
    z.extractall(ourPath)

def downloadAll():
    year = 2019
    month = 2
    while month < 13:
        url = buildUrl(year,month)
        downloadFile(url)

        month += 1
    
def makeFrame():
    ourPath = str(pathlib.Path().absolute()) + "/data/bikeShare/monthlyData/"

    onlyFiles = [f for f in listdir(ourPath) if isfile(join(ourPath, f))]

    dfs = []
    for fileName in onlyFiles:
        fileName = str(ourPath + fileName)
        print(fileName)
        pd.read_csv(fileName, encoding = "utf-8")
        dfs.append(pd.read_csv(fileName))

    # Concatenate all data into one DataFrame
    big_frame = pd.concat(dfs, ignore_index=True)
    return big_frame
    

In [44]:
downloadAll()
big_frame = makeFrame()
bikeShareLocations = pd.read_csv("data/bikeShareLocations.csv")

/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201910-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201909-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201906-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201911-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201904-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201908-capitalbikeshare-tripdata
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201903-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201912-capitalbikeshare-tripdata.csv
/User/jgluck/Documents/GitHub/dc-bike-analysis/data/bikeShare/monthlyData/201907-capitalbikeshare-tripdata
/User/jgl

In [11]:
big_frame.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,429,2019-10-01 00:01:59,2019-10-01 00:09:08,31214,17th & Corcoran St NW,31203,14th & Rhode Island Ave NW,W23731,Member
1,1935,2019-10-01 00:03:07,2019-10-01 00:35:23,31269,3rd St & Pennsylvania Ave SE,31269,3rd St & Pennsylvania Ave SE,W22377,Member
2,563,2019-10-01 00:03:51,2019-10-01 00:13:14,31214,17th & Corcoran St NW,31251,12th & L St NW,W23473,Member
3,625,2019-10-01 00:05:34,2019-10-01 00:15:59,31301,Ward Circle / American University,31317,Wisconsin Ave & Ingomar St NW,W21371,Member
4,312,2019-10-01 00:05:47,2019-10-01 00:10:59,31519,1st & O St NW,31109,7th & T St NW,W23666,Member


In [45]:
bikeShareLocations.head()

,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE,INSTALLED,LOCKED,INSTALL_DATE,REMOVAL_DATE,TEMPORARY_INSTALL,NUMBER_OF_BIKES,NUMBER_OF_EMPTY_DOCKS,X,Y,SE_ANNO_CAD_DATA,OWNER
0,302988495,438,Reston Town Center Transit Station,32214,38.957037,-77.359718,YES,NO,NaN,NaN,NO,5,12,368821.700515,143298.327458,NaN,NaN
1,302988496,439,1st & D St SE,31643,38.884916,-77.005965,YES,NO,NaN,NaN,NO,3,16,399482.463435,135230.875721,NaN,DC
2,302988497,440,Wiehle Ave & Reston Station Blvd,32215,38.949200,-77.336800,YES,NO,NaN,NaN,NO,7,11,370804.882704,142420.776929,NaN,NaN
3,302988498,441,Library St & Freedom Dr,32216,38.959633,-77.358741,YES,NO,NaN,NaN,NO,9,10,368907.513324,143586.171975,NaN,NaN
4,302988499,442,Sunset Hills Rd & Discovery Square,32217,38.955340,-77.357611,YES,NO,NaN,NaN,NO,1,14,369003.584328,143109.228805,NaN,NaN


In [46]:
startLocations = big_frame['Start station number'].values
endLocations = big_frame['End station number'].values
allLocations = [i for i in list(np.unique(np.concatenate((startLocations, endLocations), axis=0)))]
locationList = bikeShareLocations['TERMINAL_NUMBER']

In [49]:
print(len(allLocations))
print(len(locationList))

b = 0
for i in locationList:
    if (i in allLocations):
        b = b + 1
    else:
        print(i)
print(b)

31325
32901
577


In [56]:
fr = []
for ss, es in zip(big_frame['Start station number'], big_frame['End station number']):
    fr.append(str(ss) + ' - ' + str(es))
big_frame['fullRoutes'] = fr

In [90]:
start_station_ids = []
end_station_ids = []
total_rides = []
start_station_lats = []
start_station_longs = []
end_station_lats = []
end_station_longs = []

for i,j in zip(big_frame['fullRoutes'].value_counts().index,big_frame['fullRoutes'].value_counts()):
    
    start_station_id = int((i.split("-")[0]).strip())
    start_station_ids.append(start_station_id)
    end_station_id = int((i.split("-")[1]).strip())
    end_station_ids.append(end_station_id)
    
    start_station_row = bikeShareLocations[bikeShareLocations['TERMINAL_NUMBER'] == start_station_id]
    end_station_row = bikeShareLocations[bikeShareLocations['TERMINAL_NUMBER'] == end_station_id]

    if (len(start_station_row['LATITUDE'].values) > 0):
        start_station_lats.append(start_station_row['LATITUDE'].values[0])
    else:
        start_station_lats.append(None)
        
    if (len(start_station_row['LONGITUDE'].values) > 0):
        start_station_longs.append(start_station_row['LONGITUDE'].values[0])
    else:
        start_station_longs.append(None)
        
    if (len(end_station_row['LATITUDE'].values) > 0):
        end_station_lats.append(end_station_row['LATITUDE'].values[0])
    else:
        end_station_lats.append(None)
        
    if (len(end_station_row['LONGITUDE'].values) > 0):
        end_station_longs.append(end_station_row['LONGITUDE'].values[0])
    else:
        end_station_longs.append(None)
    
    total_rides.append(j)
    
bikeShareMovement = pd.DataFrame(
            {
                "start_station_id" : start_station_ids,
                "end_station_id" : end_station_ids,
                "total_ride" : total_rides,
                "start_station_lat" : start_station_lats,
                "start_station_long" : start_station_longs,
                "end_station_lat" : end_station_lats,
                "end_station_long" : end_station_longs
            })

In [103]:
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'].notna()]

In [104]:
bikeShareMovement.head()
bikeShareMovement.to_csv("data/bikeShareMovement.csv")

In [105]:
bikeShareMovement.head()

,start_station_id,end_station_id,total_ride,start_station_lat,start_station_long,end_station_lat,end_station_long
0,31248,31248,4830,38.888767,-77.028580,38.888767,-77.028580
1,31623,31615,4066,38.896960,-77.004930,38.899972,-76.998347
2,31288,31288,3426,38.890493,-77.017253,38.890493,-77.017253
3,31247,31247,3403,38.888553,-77.032429,38.888553,-77.032429
4,31258,31249,3356,38.888251,-77.049426,38.879819,-77.037413


In [106]:
bikeShareMovement['start_station_lat'].value_counts

<bound method IndexOpsMixin.value_counts of 0        38.888767
1        38.896960
2        38.890493
3        38.888553
4        38.888251
           ...    
85024    38.892275
85025    38.957463
85026    38.931991
85027    38.912644
85028    39.107709
Name: start_station_lat, Length: 84107, dtype: float64>